In [1]:
##Importing seaborn, numpy, pandas, matplotlib ##MS3 START HERE
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')
DATA="/content/drive/MyDrive/data_MS2"
#read the csv
games = pd.read_csv(f"{DATA}/games.csv")
users = pd.read_csv(f"{DATA}/users.csv")
recs = pd.read_csv(f"{DATA}/recommendations.csv")

pd.set_option('display.max_columns', None)
display(games)
display(users)
display(recs)

Mounted at /content/drive


,app_id,title,date_release,win,mac,linux,rating,positive_ratio,user_reviews,price_final,price_original,discount,steam_deck
0,13500,Prince of Persia: Warrior Within™,2008-11-21,True,False,False,Very Positive,84,2199,9.99,9.99,0.0,True
1,22364,BRINK: Agents of Change,2011-08-03,True,False,False,Positive,85,21,2.99,2.99,0.0,True
2,113020,Monaco: What's Yours Is Mine,2013-04-24,True,True,True,Very Positive,92,3722,14.99,14.99,0.0,True
3,226560,Escape Dead Island,2014-11-18,True,False,False,Mixed,61,873,14.99,14.99,0.0,True
4,249050,Dungeon of the ENDLESS™,2014-10-27,True,True,False,Very Positive,88,8784,11.99,11.99,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
50867,2296380,I Expect You To Die 3: Cog in the Machine,2023-09-28,True,False,False,Very Positive,96,101,22.00,0.00,0.0,True
50868,1272080,PAYDAY 3,2023-09-21,True,False,False,Mostly Negative,38,29458,40.00,0.00,0.0,True
50869,1402110,Eternights,2023-09-11,True,False,False,Very Positive,89,1128,30.00,0.00,0.0,True
50870,2272250,Forgive Me Father 2,2023-10-19,True,False,False,Very Positive,95,82,17.00,0.00,0.0,True


,user_id,products,reviews
0,7360263,359,0
1,14020781,156,1
2,8762579,329,4
3,4820647,176,4
4,5167327,98,2
...,...,...,...
14306059,5047430,6,0
14306060,5048153,0,0
14306061,5059205,31,0
14306062,5074363,0,0


,app_id,helpful,funny,date,is_recommended,hours,user_id,review_id
0,975370,0,0,2022-12-12,True,36.3,51580,0
1,304390,4,0,2017-02-17,False,11.5,2586,1
2,1085660,2,0,2019-11-17,True,336.5,253880,2
3,703080,0,0,2022-09-23,True,27.4,259432,3
4,526870,0,0,2021-01-10,True,7.9,23869,4
...,...,...,...,...,...,...,...,...
41154789,633230,0,0,2021-02-15,True,41.0,1606890,41154789
41154790,758870,8,0,2019-07-18,False,8.0,1786254,41154790
41154791,696170,3,10,2018-03-26,False,2.0,6370324,41154791
41154792,696170,0,0,2018-06-11,True,4.0,1044289,41154792


In [2]:
rec_cols = ["app_id","helpful","funny","date","is_recommended","hours","user_id"]

In [3]:
# ---------------------------
# Basic typing
# ---------------------------
recs["helpful"] = pd.to_numeric(recs["helpful"], errors="coerce", downcast="integer")
recs["funny"]   = pd.to_numeric(recs["funny"],   errors="coerce", downcast="integer")
recs["hours"]   = pd.to_numeric(recs["hours"],   errors="coerce")
recs["date"]    = pd.to_datetime(recs["date"],   errors="coerce")

recs["is_recommended"] = (
    recs["is_recommended"].astype(str).str.strip().str.lower()
           .map({"true":1,"1":1,"yes":1,"false":0,"0":0,"no":0})
).fillna(0).astype("int8")

In [4]:
# ---------------------------
# Lean enrichment
# ---------------------------
if users is not None and len(users):
    # downcast user numerics
    for c in ["account_age","num_friends"]:
        if c in users.columns:
            users[c] = pd.to_numeric(users[c], errors="coerce", downcast="integer")
    recs = recs.merge(users, on="user_id", how="left")

if games is not None and len(games):
    for b in ["win","mac","linux"]:
        if b in games.columns:
            games[b] = games[b].astype(str).str.lower().map({"true":1,"1":1,"yes":1,"false":0,"0":0,"no":0}).astype("float32")
    if "date_release" in games.columns:
        games["date_release"] = pd.to_datetime(games["date_release"], errors="coerce")
    # downcast handy numerics
    for c in ["price_final","positive_ratio"]:
        if c in games.columns:
            games[c] = pd.to_numeric(games[c], errors="coerce")
    recs = recs.merge(games, on="app_id", how="left")

In [5]:
# ---------------------------
# Feature expansion
# ---------------------------
df = recs  # reuse same object to avoid extra copy

df["log_hours"]   = np.log1p(df["hours"]).astype("float32")
df["log_helpful"] = np.log1p(df["helpful"]).astype("float32")
df["log_funny"]   = np.log1p(df["funny"]).astype("float32")
df["helpful_per_hour"] = (df["helpful"]/(1.0+df["hours"])).astype("float32")
df["funny_per_hour"]   = (df["funny"]  /(1.0+df["hours"])).astype("float32")
df["hf_interact"]      = (df["helpful"]*df["funny"]).astype("float32")

if "date" in df.columns:
    md = df["date"].max()
    df["recency_days"] = (md - df["date"]).dt.days.astype("float32")
    df["year"]  = df["date"].dt.year.astype("Int32")
    df["month"] = df["date"].dt.month.astype("Int8")
    df["dow"]   = df["date"].dt.dayofweek.astype("Int8")

if "date_release" in df.columns and "date" in df.columns:
    df["game_age_days"] = (df["date"] - df["date_release"]).dt.days.astype("float32")

In [6]:
# ---------------------------
# Build X, y; memory-lean encoding
# ---------------------------
y = df["is_recommended"].astype("int8")
X = df.drop(columns=[c for c in ["is_recommended","date","date_release","user_id","app_id"] if c in df.columns], errors="ignore")

# Identify categoricals
obj_cols = [c for c in X.columns if X[c].dtype == "object"]

# Frequency-encode high-cardinality; one-hot only small-cardinality (<= 20 uniques)
small_ohe = []
freq_enc  = []
for c in obj_cols:
    nun = X[c].nunique(dropna=True)
    if nun <= 20:
        small_ohe.append(c)
    else:
        freq_enc.append(c)

# frequency encoding (train-only statistics later, but for simplicity we do global here to save RAM)
for c in freq_enc:
    vc = X[c].value_counts(normalize=True)
    X[c] = X[c].map(vc).fillna(0).astype("float32")

# one-hot small categoricals (keeps feature count small); drop_first=False to be safe for tree splits
if small_ohe:
    X = pd.get_dummies(X, columns=small_ohe, drop_first=False)

# Impute numeric NaNs with median (column-wise, in-place)
for c in X.columns:
    if pd.api.types.is_numeric_dtype(X[c]):
        med = X[c].median()
        X[c] = X[c].fillna(med)

# Downcast numerics to float32 to halve memory
for c in X.columns:
    if pd.api.types.is_numeric_dtype(X[c]):
        X[c] = X[c].astype("float32")

In [7]:
# ---------------------------
# Train/Test split
# ---------------------------
rng = np.random.default_rng(42)
idx = rng.permutation(len(y))
n_test = int(0.2*len(y))
te, tr = idx[:n_test], idx[n_test:]

# Convert to compact numpy arrays (float32)
Xtr = X.iloc[tr].to_numpy(dtype=np.float32, copy=False)
Xte = X.iloc[te].to_numpy(dtype=np.float32, copy=False)
ytr = y.iloc[tr].to_numpy(dtype=np.int8, copy=False)
yte = y.iloc[te].to_numpy(dtype=np.int8, copy=False)

print("Shapes:", Xtr.shape, Xte.shape)

Shapes: (32923836, 36) (8230958, 36)


In [8]:
# ---------------------------
# CART
# ---------------------------
def gini(y):
    if len(y)==0: return 0.0
    p = np.bincount(y, minlength=2)/len(y)
    return 1.0 - (p*p).sum()

def best_split(X, y, min_leaf=50, max_candidates=32):
    n, d = X.shape
    base = gini(y); best = (None, None, None)
    for j in range(d):
        # sample candidate thresholds via quantiles on a subsample for speed/memory
        col = X[:, j]
        if n > 20000:
            samp = col[np.random.default_rng(0).choice(n, size=20000, replace=False)]
        else:
            samp = col
        vals = np.unique(samp)
        if len(vals) > max_candidates:
            qs = np.linspace(0, 1, max_candidates+2)[1:-1]
            thrs = np.quantile(vals, qs)
        else:
            thrs = (vals[:-1] + vals[1:]) / 2.0 if len(vals) > 1 else vals
        for t in thrs:
            L = col <= t
            nl = L.sum(); nr = n - nl
            if nl < min_leaf or nr < min_leaf:
                continue
            g = base - (nl/n)*gini(y[L]) - (nr/n)*gini(y[~L])
            if best[2] is None or g > best[2]:
                best = (j, float(t), float(g))
    return best

class Node:
    __slots__ = ("j","t","left","right","pred")
    def __init__(self, pred=None, j=None, t=None, left=None, right=None):
        self.pred=pred; self.j=j; self.t=t; self.left=left; self.right=right

def build_tree(X, y, depth=0, max_depth=8, min_leaf=50):
    # stop if pure/too small/depth cap
    if depth>=max_depth or len(y) < 2*min_leaf or len(np.unique(y))==1:
        return Node(pred=int(np.bincount(y, minlength=2).argmax()))
    j,t,g = best_split(X, y, min_leaf=min_leaf)
    if j is None:
        return Node(pred=int(np.bincount(y, minlength=2).argmax()))
    L = X[:, j] <= t
    left  = build_tree(X[L],  y[L],  depth+1, max_depth, min_leaf)
    right = build_tree(X[~L], y[~L], depth+1, max_depth, min_leaf)
    return Node(j=j, t=t, left=left, right=right)

def predict_tree(root, X):
    out = np.empty(X.shape[0], dtype=np.int8)
    for i in range(X.shape[0]):
        n = root
        while n.pred is None:
            n = n.left if X[i, n.j] <= n.t else n.right
        out[i] = n.pred
    return out

In [9]:
# ---------------------------
# Train + Evaluate (keep arrays float32/int8)
# ---------------------------
tree = build_tree(Xtr, ytr, max_depth=10, min_leaf=100)
yhat_tr = predict_tree(tree, Xtr)
yhat_te = predict_tree(tree, Xte)

acc_tr = float((yhat_tr == ytr).mean())
acc_te = float((yhat_te == yte).mean())

def cm(y_true, y_pred):
    TN = int(((y_true==0)&(y_pred==0)).sum())
    FP = int(((y_true==0)&(y_pred==1)).sum())
    FN = int(((y_true==1)&(y_pred==0)).sum())
    TP = int(((y_true==1)&(y_pred==1)).sum())
    return np.array([[TN, FP],[FN, TP]], dtype=int)

print("=== Decision Tree (memory-lean) ===")
print("Train accuracy:", round(acc_tr,4))
print("Test  accuracy:", round(acc_te,4))
print("Confusion matrix (test):\n", cm(yte, yhat_te))
print("\nTips: If still OOM, drop very wide text columns (e.g., 'genres'), or raise min_leaf/max_candidates to shrink the tree search.")

=== Decision Tree (memory-lean) ===
Train accuracy: 0.8756
Test  accuracy: 0.8755
Confusion matrix (test):
 [[ 303353  867169]
 [ 157213 6903223]]

Tips: If still OOM, drop very wide text columns (e.g., 'genres'), or raise min_leaf/max_candidates to shrink the tree search.


## Where does my model fit?

I tried different hyperparams:

* **Shallow tree** (`max_depth=3`) → both train/test were low → underfitting.
* **Deep tree** (`max_depth=20`) → train super high, test dropped → overfitting.
* **Balanced tree** (`max_depth=10, min_leaf=100`) → train a bit higher than test but both solid. This sits in the “just right” middle, so I picked this one.

---

## What’s next?

* Maybe a **linear/logistic baseline** just to compare.

---

## Conclusion (1st model)

The first balanced tree actually did pretty well, decent accuracy and not crazy overfit. To push it further I’d:

* add better features,
* tune depth/leaf more,
* handle imbalance if needed,
* and most importantly try ensembles (RF or boosting).

**Bottom line:** single tree is a good start, but ensembles will almost always beat it.